In [1]:
import torch
from diffusers import StableDiffusionPipeline

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
torch_dtype=torch.float16 if device == "cuda" else torch.float32
# Load the pipeline (no need for scheduler, text encoder, etc.)
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch_dtype
)
pipe = pipe.to(device)
_ = pipe.vae.eval()

2025-04-18 07:59:26.846914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744977566.858321    8329 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744977566.861763    8329 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 07:59:26.875284: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Could not load bitsandbytes native library: 'NoneType' object has no attribute 'split'
Traceback (most recent call last):
  F

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:

latents = torch.randn(1, 4, 64, 64, device=device, dtype = torch_dtype, requires_grad=True)

with torch.enable_grad():
    decoded = pipe.vae.decode(latents)  # Returns SampleOutput
    image = decoded.sample  # This may be detached

# Check if gradients flow
scalar = image.sum()
scalar.backward()

print("Gradient of latents (should not be None):", latents.grad)

Gradient of latents (should not be None): tensor([[[[ 24.0160,  17.1888,  16.1042,  ...,  18.8757,  18.0343,  19.4322],
          [ 23.0236,  18.8044,  15.4070,  ...,  17.8581,  13.1207,  21.1108],
          [ 16.7784,  18.2736,  19.2088,  ...,  51.8460, -14.1167,  17.5220],
          ...,
          [ 22.9365,   8.2854,  18.9523,  ...,  19.0432,  19.7006,  19.5057],
          [ 20.4679,  21.6788,  11.6978,  ...,  18.4686,  13.2063,   9.5370],
          [ 15.4416,  24.6368,  18.8424,  ...,  13.3929,  21.0996,  14.3285]],

         [[ 10.8740,  11.5339,  13.3749,  ...,   8.1413,  15.2031,  11.4969],
          [ 11.6101,  13.9532,  12.2978,  ...,   9.2128,   8.2065,  13.7141],
          [ 14.3990,  16.4335,  13.8345,  ...,   9.2702,  24.3216,  13.3538],
          ...,
          [ 15.3893,  15.3023,  16.0283,  ...,  14.5434,  14.8369,  14.0364],
          [ 11.4864,  21.0988,   9.4449,  ...,  14.3880,  10.0669,  10.4385],
          [ 19.8336,  16.6974,  15.6520,  ...,  14.9978,   7.4506,  